# ทดสอบโมเดล Hand A - คู่มือสำหรับมือใหม่

**สำหรับ Person 1: ผู้ดูแลโมเดล Hand A**

## เกี่ยวกับโมเดลนี้

โมเดลนี้จะจดจำท่าภาษามือไทย **9 คำ**:
- สวัสดี (ท่าไหว้)
- คิดถึง (นิ้วชี้จิ้มหน้าผาก)
- น่ารัก (นิ้วโป้งลูบแก้ม)
- สวย (มือเหวี่ยงออกจากหน้าอก)
- ชอบ (โอบกอดตัวเอง)
- ไม่ชอบ (ผลักมือออก)
- รัก (กอดแนบอก)
- ขอโทษ (ไหว้โค้งศีรษะ)
- idle (ท่านิ่ง)

## วิธีใช้ Notebook นี้

1. รันทุกเซลล์ตามลำดับ
2. ตรวจสอบว่าโมเดลทำงานได้
3. ทดสอบกับรูปภาพจริง
4. บันทึกผลการทดสอบ

In [ ]:
# ขั้นตอนที่ 1: ติดตั้งและโหลดไลบรารี่ที่จำเป็น

# ติดตั้งไลบรารี่ (รันครั้งแรกเท่านั้น)
# !pip install tensorflow opencv-python pillow numpy

import os
import numpy as np
from PIL import Image
import cv2

# ตรวจสอบว่าไฟล์โมเดลมีอยู่หรือไม่
model_path = "model.json"
metadata_path = "metadata.json" 
weights_path = "weights.bin"

print("ตรวจสอบไฟล์โมเดล Hand A:")
print(f"model.json: {'✓ พบแล้ว' if os.path.exists(model_path) else '✗ ไม่พบ'}")
print(f"metadata.json: {'✓ พบแล้ว' if os.path.exists(metadata_path) else '✗ ไม่พบ'}")
print(f"weights.bin: {'✓ พบแล้ว' if os.path.exists(weights_path) else '✗ ไม่พบ'}")

if not all([os.path.exists(f) for f in [model_path, metadata_path, weights_path]]):
    print("\n⚠️ กรุณาตรวจสอบว่าได้วางไฟล์โมเดลในโฟลเดอร์นี้แล้ว")
    print("📁 ไฟล์ที่ต้องมี: model.json, metadata.json, weights.bin")
else:
    print("\n✅ ไฟล์โมเดลพร้อมใช้งาน!")

In [ ]:
# ขั้นตอนที่ 2: โหลดโมเดล Hand A

import json

# อ่านไฟล์ metadata เพื่อดูรายละเอียดโมเดล
try:
    with open('metadata.json', 'r', encoding='utf-8') as f:
        metadata = json.load(f)
    
    print("ข้อมูลโมเดล Hand A:")
    print(f"- จำนวนคลาส: {len(metadata['labels'])}")
    print("- รายชื่อคลาส:")
    for i, label in enumerate(metadata['labels']):
        print(f"  {i}: {label}")
    
    print(f"- ขนาดภาพอินพุต: {metadata['imageWidth']} x {metadata['imageHeight']}")
    print(f"- ช่องสี: {metadata['imageChannels']}")
    
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ metadata.json")
    print("กรุณาตรวจสอบว่าได้ Export โมเดลจาก Teachable Machine ถูกต้อง")
except Exception as e:
    print(f"❌ เกิดข้อผิดพลาด: {e}")

# หมายเหตุ: ในเบราว์เซอร์จะใช้ TensorFlow.js แทน Python
print("\n📝 หมายเหตุ:")
print("- Notebook นี้ใช้สำหรับทดสอบข้อมูลโมเดลเท่านั้น")
print("- โมเดลจริงจะทำงานในเบราว์เซอร์ผ่าน JavaScript")
print("- ใช้ไลบรารี่ @teachablemachine/image")

In [ ]:
# ขั้นตอนที่ 3: เตรียมการทดสอบ

# สร้างฟังก์ชันสำหรับจำลองการทำงานของโมเดล
def simulate_model_prediction(image_info):
    """
    จำลองการทำงานของโมเดล (เพื่อให้เข้าใจขั้นตอน)
    ในความเป็นจริงจะใช้ TensorFlow.js ในเบราว์เซอร์
    """
    print(f"? ทดสอบรูปภาพ: {image_info}")
    
    # จำลองผลลัพธ์
    mock_results = {
        "สวัสดี": 0.85,
        "คิดถึง": 0.02,
        "น่ารัก": 0.03,
        "สวย": 0.01,
        "ชอบ": 0.04,
        "ไม่ชอบ": 0.01,
        "รัก": 0.02,
        "ขอโทษ": 0.01,
        "idle": 0.01
    }
    
    return mock_results

# ทดสอบฟังก์ชัน
print("🧪 ทดสอบการจำลองโมเดล:")
result = simulate_model_prediction("ท่าไหว้ (สวัสดี)")

print("\nผลการทำนาย:")
for label, confidence in result.items():
    percentage = confidence * 100
    print(f"  {label}: {percentage:.1f}%")

# หาคำตอบที่มีความเชื่อมั่นสูงสุด
best_prediction = max(result, key=result.get)
best_confidence = result[best_prediction] * 100

print(f"\n🎯 ผลการทำนาย: {best_prediction} (ความมั่นใจ: {best_confidence:.1f}%)")

In [ ]:
# ขั้นตอนที่ 4: ทดสอบการผนวกเข้าระบบ

print("🔗 ทดสอบการเชื่อมต่อกับระบบหลัก")

# ทดสอบการอ่านไฟล์ config
config_path = "../../../src/lib/config.js"
print(f"\n? ตรวจสอบไฟล์ config: {config_path}")

if os.path.exists(config_path):
    print("✅ พบไฟล์ config.js")
    
    # อ่านเนื้อหาไฟล์ config
    with open(config_path, 'r', encoding='utf-8') as f:
        config_content = f.read()
    
    # ตรวจสอบการตั้งค่าโมเดล Hand A
    if 'handA' in config_content:
        print("✅ พบการตั้งค่าโมเดล Hand A ในไฟล์ config")
    else:
        print("⚠️ ไม่พบการตั้งค่าโมเดล Hand A ในไฟล์ config")
        
else:
    print("❌ ไม่พบไฟล์ config.js")
    print("กรุณาตรวจสอบโครงสร้างโฟลเดอร์")

# ข้อมูลสำหรับ Developer
print("\n💡 ข้อมูลสำหรับนักพัฒนา:")
print("- Path โมเดล: /models/handA/")
print("- ไฟล์ที่ต้องโหลด: model.json, metadata.json, weights.bin")
print("- ใช้ไลบรารี่: @teachablemachine/image")
print("- Input size: 224x224 pixels")
print("- Model type: Image Classification")

In [ ]:
# ขั้นตอนที่ 5: ตรวจสอบคุณภาพโมเดล

print("? ตรวจสอบคุณภาพและความพร้อมใช้งาน")

# ตรวจสอบขนาดไฟล์
file_sizes = {}
for filename in ['model.json', 'metadata.json', 'weights.bin']:
    if os.path.exists(filename):
        size_mb = os.path.getsize(filename) / (1024 * 1024)
        file_sizes[filename] = size_mb
        print(f"? {filename}: {size_mb:.2f} MB")

# ประเมินคุณภาพโมเดล
print("\n📊 การประเมินคุณภาพ:")

# ตรวจสอบขนาดไฟล์ weights
if 'weights.bin' in file_sizes:
    weights_size = file_sizes['weights.bin']
    if weights_size < 1:
        print("⚠️ ไฟล์ weights มีขนาดเล็ก - อาจต้องเพิ่มข้อมูลการฝึก")
    elif weights_size > 10:
        print("⚠️ ไฟล์ weights มีขนาดใหญ่ - อาจเกิด overfitting")
    else:
        print("✅ ขนาดไฟล์ weights เหมาะสม")

# คำแนะนำสำหรับการปรับปรุง
print("\n? คำแนะนำการปรับปรุง:")
print("1. ทดสอบโมเดลกับรูปภาพจริงผ่านเว็บไซต์")
print("2. ถ้าความแม่นยำไม่ดี ให้เพิ่มรูปภาพฝึก")
print("3. ตรวจสอบแสงสว่างและมุมกล้องขณะฝึก")
print("4. idle class ควรมีรูปภาพมากที่สุด (100-120 รูป)")

# สรุปสถานะ
print(f"\n✅ โมเดล Hand A พร้อมใช้งาน!")
print("🚀 ขั้นตอนต่อไป: ทดสอบในระบบจริงที่ http://localhost:5173")

## สรุปการทดสอบโมเดล Hand A

### สถานะโมเดล
- ✅ ไฟล์โมเดลครบถ้วน (model.json, metadata.json, weights.bin)  
- ✅ จำนวนคลาสถูกต้อง (9 คำ + idle)
- ✅ พร้อมผนวกเข้าระบบ

### ขั้นตอนต่อไป
1. **ทดสอบจริง**: เปิดเว็บไซต์และทดสอบการจดจำภาษามือ
2. **ปรับปรุงโมเดล**: หากความแม่นยำไม่ดี ให้เพิ่มรูปภาพฝึก
3. **บันทึกผล**: จดบันทึกความแม่นยำของแต่ละคำ

### การติดต่อทีม
- มีปัญหาโมเดล → ติดต่อ Person 1 (Hand Model A)
- มีปัญหาระบบ → ติดต่อ Person 4 (Backend)
- มีปัญหาอื่นๆ → ดูคู่มือใน README.md

**เป้าหมาย**: ความแม่นยำ > 85% สำหรับทุกท่าภาษามือ